# Download Data

In [1]:
!pip3 install delta-sharing --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.7/72.7 MB 6.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import delta_sharing
import json
import os
from google.colab import userdata

Please be sure the credentials names are the same
Tutorial:
1. In colab go to Secretes and add the desire credentials

More Info:
[Info](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75)

In [3]:
credentials = {
	"shareCredentialsVersion": 1,
	"bearerToken": userdata.get('bearer'),
	"endpoint": userdata.get('endpoint'),
	"expirationTime": "2024-11-12T04:53:07.255Z"
}

In [4]:
file_name = "config.share"

# Write the dictionary to a JSON file in a single line
with open(file_name, 'w') as json_file:
    json.dump(credentials, json_file, separators=(',', ':'), ensure_ascii=False)

print(f"Dictionary saved as a single-line JSON in {file_name}")

Dictionary saved as a single-line JSON in config.share


In [5]:
# Point to the profile file. It can be a file on the local file system or a file on a remote storage.
profile_file = file_name

# Create a SharingClient.
client = delta_sharing.SharingClient(profile_file)

# List all shared tables.
client.list_all_tables()

[Table(name='events_1percent', share='unlock-share', schema='bronze'),
 Table(name='events', share='unlock-share', schema='bronze'),
 Table(name='events_001percent', share='unlock-share', schema='bronze'),
 Table(name='events_10percent', share='unlock-share', schema='bronze'),
 Table(name='events_01percent', share='unlock-share', schema='bronze')]

In [6]:
# Create a url to access a shared table.
# A table path is the profile file path following with `#` and the fully qualified name of a table
# (`<share-name>.<schema-name>.<table-name>`).
table_url = profile_file + "#unlock-share.bronze.events_001percent" # -> cambien el porcentaje de acuerdo a la lista de arriba

In [7]:
# Load a table as a Pandas DataFrame. This can be used to process tables that can fit in the memory.
df = delta_sharing.load_as_pandas(table_url)

In [ ]:
df.head(1)

,globaleventid,sqldate,monthyear,year,fractiondate,actor1code,actor1name,actor1countrycode,actor1knowngroupcode,actor1ethniccode,...,actor2geo_featureid,actiongeo_type,actiongeo_fullname,actiongeo_countrycode,actiongeo_adm1code,actiongeo_lat,actiongeo_long,actiongeo_featureid,dateadded,sourceurl
0,1120582762,20230813,202308,2023,2023.6110,USA,UNITED STATES,USA,None,None,...,None,2,"Arkansas, United States",US,USAR,34.9513,-92.3809,AR,20230813,https://www.eldoradonews.com/news/2023/aug/13/...


# Web pages classification


In [8]:
import pandas as pd
import requests
import threading

In [9]:
urls: pd.Series = df.sourceurl  ## Url column

In [ ]:
# Shared resources
connections = []
errors = []

# Lock for synchronizing access to shared resources
lock = threading.Lock()

def worker(url, index, total_urls):
    try:
        request = requests.get(url, timeout=1)
        with lock:  # Ensure thread-safe access to shared lists
            connections.append((request.status_code, url))
    except Exception as e:
        with lock:  # Ensure thread-safe access to shared lists
            errors.append(url)
    #finally:
        #print(f"classified [{index} of {total_urls}]", url) # Uncommend this to get log about classification

def fetch_status_codes(urls) -> tuple:
    global connections, errors
    connections = []  # Clear previous results
    errors = []

    # Create and start threads
    threads = []
    for index, url in enumerate(urls, start=1):
        thread = threading.Thread(target=worker, args=(url, index, len(urls)))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    return connections, errors


# Fetch status codes and errors
status_codes, error_urls = fetch_status_codes(urls)

## Classification with Time
If you want to know how many time it takes to classify urls run this

In [10]:
import threading
import requests
import time

# Shared resources
connections = []
errors = []

# Lock for synchronizing access to shared resources
lock = threading.Lock()

def worker(url, index, total_urls):
    try:
        request = requests.get(url, timeout=1)
        with lock:  # Ensure thread-safe access to shared lists
            connections.append((request.status_code, url))
    except Exception as e:
        with lock:  # Ensure thread-safe access to shared lists
            errors.append(url)
    #finally:
        #print(f"classified [{index} of {total_urls}]", url) # Uncomment this to get log about classification

def fetch_status_codes(urls) -> tuple:
    global connections, errors
    connections = []  # Clear previous results
    errors = []

    # Create and start threads
    threads = []
    for index, url in enumerate(urls, start=1):
        thread = threading.Thread(target=worker, args=(url, index, len(urls)))
        thread.start()
        threads.append(thread)

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    return connections, errors

# Measure execution time
start_time = time.time()  # Record start time

# Fetch status codes and errors
status_codes, error_urls = fetch_status_codes(urls)

end_time = time.time()  # Record end time
elapsed_time = end_time - start_time  # Calculate elapsed time

# Print results
#print(f"Status Codes: {status_codes}")
#print(f"Error URLs: {error_urls}")
print(f"Execution Time: {elapsed_time:.2f} seconds")


Execution Time: 52.03 seconds
